In [ ]:
import json
import requests
import time
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
YEAR = 2019
BASE_URL_COMPANY_GET = "https://fortune.com/worlds-most-admired-companies/2020/apple/" # should be rank 1

## get companies

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
companies = []
driver.get(BASE_URL_COMPANY_GET)
companies.append(driver.current_url.split('/')[-2])
for x in range(1, 1000):
    time.sleep(random.randint(0,3))
    button = driver.find_element_by_class_name('singlePagination__next2--3D89W')
    button.click()
    companies.append(driver.current_url.split('/')[-2])

## get data

In [ ]:
baseurl = "https://fortune.com/worlds-most-admired-companies/{}/{}/"
companydict = {}
for c in companies:
    response = requests.get(baseurl.format(YEAR, c))
    soup = BeautifulSoup(response.content, 'html.parser')
    soupmetrics = soup.find_all('tr')
    metricdict = {}
    for s in soupmetrics:
        metricdict[s.find_all('td')[0].text] = s.find_all('td')[1].text
    companydict[soup.find_all('title')[0].text.split(' |')[0]] = metricdict

In [ ]:
with open(f'../data/fortune/f500_reputation_{YEAR}.json', 'w') as outfile:
    json.dump(companydict, outfile)

## to pandas

In [ ]:
import csv
import json
import pandas as pd

In [ ]:
with open(f'../data/fortune/f500_reputation_{YEAR}.json', 'r') as infile:
    companydict = json.load(infile)

In [ ]:
frames = []
for k, v in companydict.items():
    dfsub = pd.DataFrame({'company': [k]})
    for subk, subv in v.items():
        dfsub[subk] = subv
    frames.append(dfsub)
dfcompany = pd.concat(frames)
dfcompany = dfcompany.reset_index(drop=True)

In [ ]:
dfcompany = dfcompany.loc[lambda x: ~x['Overall Score'].isnull()]

In [ ]:
dfcompany.to_csv(f'../data/fortune/f500_reputation_{YEAR}.csv',
                 index=False,
                 quoting=csv.QUOTE_NONNUMERIC,
                 quotechar='"')